# Model Training: Baselines vs MLP

This notebook demonstrates training and comparing baseline models with multi-layer perceptron (MLP) for cfDNA cancer detection.

## Objectives:
1. Train baseline models (Logistic Regression, Random Forest)
2. Train MLP model with hyperparameter tuning
3. Compare model performance
4. Analyze results and generate visualizations
5. Demonstrate nested cross-validation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import subprocess
import json

# Add src to path
sys.path.append('../src')

from cfdna.metrics import comprehensive_evaluation, compare_models, delong_test
from cfdna.viz import plot_roc_curve, plot_precision_recall_curve

plt.style.use('seaborn-v0_8')
sns.set_palette('husl')
%matplotlib inline

np.random.seed(42)

## 1. Train Models Using CLI

We'll use the command-line interface to train different models.

In [ ]:
# Create artifacts directory
artifacts_dir = Path('../artifacts')
artifacts_dir.mkdir(exist_ok=True)

# List of models to train
models_to_train = ['logistic_l1', 'logistic_l2', 'random_forest', 'mlp']

print("Training models using CLI...")
print("=" * 40)

training_results = {}

for model_name in models_to_train:
    print(f"\nTraining {model_name}...")
    
    # Run training command
    cmd = [
        'python', '-m', 'cfdna.train',
        '../data/synthetic_config.yaml',
        model_name,
        str(artifacts_dir),
        '--seed', '42'
    ]
    
    if model_name in ['logistic_l1', 'logistic_l2']:  # Add calibration for baselines
        cmd.append('--calibrate')
    
    result = subprocess.run(cmd, capture_output=True, text=True, cwd='../src')
    
    if result.returncode == 0:
        print(f"  ✓ {model_name} training completed")
        # Extract AUROC from output
        lines = result.stdout.split('\n')
        for line in lines:
            if 'Test AUROC:' in line:
                auroc_text = line.split('Test AUROC:')[1].strip()
                auroc_value = float(auroc_text.split()[0])
                training_results[model_name] = auroc_value
                print(f"    Test AUROC: {auroc_value:.3f}")
                break
    else:
        print(f"  ✗ {model_name} training failed:")
        print(f"    {result.stderr}")
        training_results[model_name] = None

print("\nTraining Summary:")
for model, auroc in training_results.items():
    if auroc is not None:
        print(f"  {model}: {auroc:.3f}")
    else:
        print(f"  {model}: Failed")

## 2. Load and Compare Results

Load the saved model results and create comprehensive comparisons.

In [ ]:
# Load model results
print("Loading model results...")

model_results = {}
model_predictions = {}

for model_name in models_to_train:
    results_file = artifacts_dir / f"{model_name}_results.json"
    predictions_file = artifacts_dir / f"{model_name}_test_predictions.csv"
    
    if results_file.exists():
        with open(results_file) as f:
            model_results[model_name] = json.load(f)
        print(f"  ✓ Loaded {model_name} results")
    else:
        print(f"  ✗ {model_name} results not found")
    
    if predictions_file.exists():
        model_predictions[model_name] = pd.read_csv(predictions_file)
        print(f"  ✓ Loaded {model_name} predictions")

print(f"\nLoaded results for {len(model_results)} models")

In [ ]:
# Create comparison table
if model_results:
    # Extract test set results for comparison
    test_results = {}
    for model_name, results in model_results.items():
        if 'test' in results:
            test_results[model_name] = results['test']
    
    if test_results:
        comparison_df = compare_models(test_results)
        print("Model Performance Comparison (Test Set):")
        print("=" * 50)
        print(comparison_df.to_string(index=False))
    else:
        print("No test results found for comparison")
else:
    print("No model results loaded")

## 3. Visualize Model Performance

Create comprehensive visualizations comparing model performance.

In [ ]:
# Performance comparison plots
if len(test_results) >= 2:
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 12))
    
    # AUROC comparison
    models = list(test_results.keys())
    aurocs = [test_results[model]['auroc']['mean'] for model in models]
    auroc_cis = [(test_results[model]['auroc']['ci_lower'], 
                  test_results[model]['auroc']['ci_upper']) for model in models]
    
    colors = plt.cm.Set2(np.linspace(0, 1, len(models)))
    bars1 = ax1.bar(models, aurocs, color=colors, alpha=0.7)
    
    # Add error bars
    ci_lower = [aurocs[i] - auroc_cis[i][0] for i in range(len(models))]
    ci_upper = [auroc_cis[i][1] - aurocs[i] for i in range(len(models))]
    ax1.errorbar(models, aurocs, yerr=[ci_lower, ci_upper], fmt='none', color='black', capsize=5)
    
    ax1.set_ylabel('AUROC')
    ax1.set_title('Model AUROC Comparison')
    ax1.set_ylim(0.5, 1.0)
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(True, alpha=0.3)
    
    # Add value labels
    for bar, auroc in zip(bars1, aurocs):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{auroc:.3f}', ha='center', va='bottom')
    
    # AUPRC comparison
    auprcs = [test_results[model]['auprc']['mean'] for model in models]
    auprc_cis = [(test_results[model]['auprc']['ci_lower'], 
                  test_results[model]['auprc']['ci_upper']) for model in models]
    
    bars2 = ax2.bar(models, auprcs, color=colors, alpha=0.7)
    
    ci_lower = [auprcs[i] - auprc_cis[i][0] for i in range(len(models))]
    ci_upper = [auprc_cis[i][1] - auprcs[i] for i in range(len(models))]
    ax2.errorbar(models, auprcs, yerr=[ci_lower, ci_upper], fmt='none', color='black', capsize=5)
    
    ax2.set_ylabel('AUPRC')
    ax2.set_title('Model AUPRC Comparison')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(True, alpha=0.3)
    
    for bar, auprc in zip(bars2, auprcs):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{auprc:.3f}', ha='center', va='bottom')
    
    # Calibration comparison (Brier Score)
    brier_scores = [test_results[model]['calibration']['brier_score'] for model in models]
    bars3 = ax3.bar(models, brier_scores, color=colors, alpha=0.7)
    ax3.set_ylabel('Brier Score')
    ax3.set_title('Calibration Quality (Brier Score)')
    ax3.tick_params(axis='x', rotation=45)
    ax3.grid(True, alpha=0.3)
    
    for bar, brier in zip(bars3, brier_scores):
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{brier:.3f}', ha='center', va='bottom')
    
    # Clinical utility (Sensitivity at 90% specificity)
    sens_90 = [test_results[model]['sens_at_spec90']['sensitivity'] for model in models]
    bars4 = ax4.bar(models, sens_90, color=colors, alpha=0.7)
    ax4.set_ylabel('Sensitivity')
    ax4.set_title('Sensitivity at 90% Specificity')
    ax4.tick_params(axis='x', rotation=45)
    ax4.grid(True, alpha=0.3)
    
    for bar, sens in zip(bars4, sens_90):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{sens:.3f}', ha='center', va='bottom')
    
    plt.suptitle('Comprehensive Model Performance Comparison', fontsize=16)
    plt.tight_layout()
    plt.show()
else:
    print("Insufficient models for comparison plots")

## 4. ROC and Precision-Recall Curves

Compare ROC and PR curves across models.

In [ ]:
# ROC and PR curves comparison
if model_predictions:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    colors = plt.cm.Set2(np.linspace(0, 1, len(model_predictions)))
    
    for i, (model_name, pred_df) in enumerate(model_predictions.items()):
        y_true = pred_df['true_label'].values
        y_score = pred_df['predicted_prob'].values
        
        # ROC curve
        from sklearn.metrics import roc_curve, auc, precision_recall_curve
        
        fpr, tpr, _ = roc_curve(y_true, y_score)
        roc_auc = auc(fpr, tpr)
        
        ax1.plot(fpr, tpr, color=colors[i], linewidth=2,
                label=f'{model_name} (AUC = {roc_auc:.3f})')
        
        # PR curve
        precision, recall, _ = precision_recall_curve(y_true, y_score)
        pr_auc = auc(recall, precision)
        
        ax2.plot(recall, precision, color=colors[i], linewidth=2,
                label=f'{model_name} (AUC = {pr_auc:.3f})')
    
    # ROC plot formatting
    ax1.plot([0, 1], [0, 1], 'k--', alpha=0.5)
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')
    ax1.set_title('ROC Curves Comparison')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # PR plot formatting
    baseline = np.mean(y_true)
    ax2.axhline(y=baseline, color='k', linestyle='--', alpha=0.5, 
               label=f'Baseline ({baseline:.3f})')
    ax2.set_xlabel('Recall')
    ax2.set_ylabel('Precision')
    ax2.set_title('Precision-Recall Curves Comparison')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No prediction data available for curve plots")

## 5. Statistical Comparisons

Perform DeLong tests to compare model performance statistically.

In [ ]:
# DeLong statistical comparisons
if len(model_predictions) >= 2:
    print("Statistical Model Comparisons (DeLong Test)")
    print("=" * 50)
    
    model_names = list(model_predictions.keys())
    
    comparison_results = []
    
    for i, model1 in enumerate(model_names):
        for j, model2 in enumerate(model_names[i+1:], i+1):
            # Get predictions
            pred1 = model_predictions[model1]
            pred2 = model_predictions[model2]
            
            # Ensure same samples
            common_samples = set(pred1['sample_id']) & set(pred2['sample_id'])
            
            if len(common_samples) > 0:
                pred1_common = pred1[pred1['sample_id'].isin(common_samples)].sort_values('sample_id')
                pred2_common = pred2[pred2['sample_id'].isin(common_samples)].sort_values('sample_id')
                
                y_true = pred1_common['true_label'].values
                y_score1 = pred1_common['predicted_prob'].values
                y_score2 = pred2_common['predicted_prob'].values
                
                # DeLong test
                delong_result = delong_test(y_true, y_score1, y_score2)
                
                comparison_results.append({
                    'Model 1': model1,
                    'Model 2': model2,
                    'AUROC 1': delong_result['auc1'],
                    'AUROC 2': delong_result['auc2'],
                    'Difference': delong_result['auc_diff'],
                    'p-value': delong_result['p_value'],
                    'Significant': delong_result['significant']
                })
                
                significance = "***" if delong_result['p_value'] < 0.001 else \
                              "**" if delong_result['p_value'] < 0.01 else \
                              "*" if delong_result['p_value'] < 0.05 else "ns"
                
                print(f"{model1} vs {model2}:")
                print(f"  AUROC difference: {delong_result['auc_diff']:+.4f}")
                print(f"  p-value: {delong_result['p_value']:.4f} ({significance})")
                print()
    
    # Create comparison DataFrame
    if comparison_results:
        comparison_df = pd.DataFrame(comparison_results)
        print("\nComparison Summary:")
        print(comparison_df.round(4))
else:
    print("Need at least 2 models for statistical comparison")

## 6. Model Interpretation

Extract and visualize feature importance for interpretable models.

In [ ]:
# Feature importance analysis (for baseline models)
print("Feature Importance Analysis")
print("=" * 35)

# Load feature data to get feature names
sys.path.append('../src')
from cfdna.features import prepare_features

try:
    # Load configuration
    import yaml
    with open('../data/synthetic_config.yaml') as f:
        config = yaml.safe_load(f)
    
    # Prepare features to get feature names
    data = prepare_features(Path('../data'), config)
    feature_names = data['X'].columns.tolist()
    
    # For demonstration, create mock feature importance scores
    # In practice, these would be extracted from trained models
    
    # Mock importance scores (would normally load from model artifacts)
    np.random.seed(42)
    
    # Simulate methylation features being more important
    meth_features = [f for f in feature_names if f.startswith('dmr_')]
    frag_features = [f for f in feature_names if not f.startswith('dmr_')]
    
    importance_scores = {}
    
    # Higher importance for methylation features
    for feature in meth_features:
        importance_scores[feature] = np.random.exponential(0.5)
    
    # Lower importance for fragmentomics features
    for feature in frag_features:
        importance_scores[feature] = np.random.exponential(0.2)
    
    # Sort and get top features
    sorted_features = sorted(importance_scores.items(), key=lambda x: x[1], reverse=True)
    top_features = sorted_features[:15]
    
    print(f"Top 15 Most Important Features:")
    for i, (feature, score) in enumerate(top_features, 1):
        feature_type = "Methylation" if feature.startswith('dmr_') else "Fragmentomics"
        print(f"  {i:2d}. {feature:<20} ({feature_type:<13}): {score:.3f}")
    
    # Visualize feature importance
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Top features bar plot
    features, scores = zip(*top_features)
    colors = ['lightcoral' if f.startswith('dmr_') else 'lightblue' for f in features]
    
    y_pos = np.arange(len(features))
    ax1.barh(y_pos, scores, color=colors)
    ax1.set_yticks(y_pos)
    ax1.set_yticklabels(features)
    ax1.invert_yaxis()
    ax1.set_xlabel('Importance Score')
    ax1.set_title('Top 15 Feature Importances')
    ax1.grid(True, alpha=0.3, axis='x')
    
    # Feature type comparison
    meth_scores = [score for feature, score in importance_scores.items() if feature.startswith('dmr_')]
    frag_scores = [score for feature, score in importance_scores.items() if not feature.startswith('dmr_')]
    
    ax2.boxplot([meth_scores, frag_scores], labels=['Methylation', 'Fragmentomics'])
    ax2.set_ylabel('Importance Score')
    ax2.set_title('Feature Importance by Type')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print(f"\nFeature Type Summary:")
    print(f"  Methylation features: {len(meth_scores)} (mean importance: {np.mean(meth_scores):.3f})")
    print(f"  Fragmentomics features: {len(frag_scores)} (mean importance: {np.mean(frag_scores):.3f})")
    
except Exception as e:
    print(f"Error in feature importance analysis: {e}")
    print("This would normally load actual feature importance from trained models")

## 7. Generate Evaluation Report

Use the CLI to generate a comprehensive evaluation report.

In [ ]:
# Generate evaluation report using CLI
print("Generating comprehensive evaluation report...")

report_path = artifacts_dir / "evaluation_report.md"

cmd = [
    'python', '-m', 'cfdna.eval',
    str(artifacts_dir),
    str(report_path),
    '--models'] + models_to_train

result = subprocess.run(cmd, capture_output=True, text=True, cwd='../src')

if result.returncode == 0:
    print("✓ Evaluation report generated successfully")
    print(result.stdout)
    
    # Display part of the report
    if report_path.exists():
        with open(report_path) as f:
            report_content = f.read()
        
        print("\n" + "="*60)
        print("EVALUATION REPORT PREVIEW")
        print("="*60)
        # Show first 1000 characters
        print(report_content[:1000] + "..." if len(report_content) > 1000 else report_content)
        print("\n" + "="*60)
        print(f"Full report saved to: {report_path}")
        print("="*60)
else:
    print("✗ Report generation failed:")
    print(result.stderr)

## 8. Summary and Next Steps

### Key Findings:

1. **Model Performance**: Compare AUROC and AUPRC across different approaches
2. **Statistical Significance**: DeLong tests show whether performance differences are significant
3. **Calibration Quality**: Brier scores indicate how well-calibrated the probability predictions are
4. **Clinical Utility**: Sensitivity at high specificity shows practical clinical value
5. **Feature Importance**: Methylation vs fragmentomics contribution to predictions

### Model Selection Criteria:

- **Best Overall Performance**: Highest AUROC with tight confidence intervals
- **Clinical Utility**: High sensitivity at 90-95% specificity
- **Calibration**: Low Brier score for reliable probability estimates
- **Interpretability**: Feature importance aligns with biological expectations

### Next Steps:

1. **Hyperparameter Optimization**: Further tune the best-performing model
2. **Ensemble Methods**: Combine complementary models
3. **External Validation**: Test on independent datasets
4. **Clinical Translation**: Develop decision support tools
5. **Regulatory Preparation**: Document for FDA submission

**Next notebook**: `04_stats_and_calibration.ipynb` - Deep dive into statistical validation and calibration analysis